# Taller 1 Conceptos básicos

In [75]:
!myenv\Scripts\activate


c:\Users\guill\Documents\GitHub\Intro-IA-Generativa-Taller-1-Conceptos-Basicos>() 


In [129]:
!pip install -r requirements.txt

In [126]:
import nltk
import spacy
import json
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from transformers import pipeline, set_seed
import evaluate
from datasets import load_dataset
from evaluate import evaluator

In [78]:
nltk.download('all')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download("omw-1.4")

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\guill\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\guill\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\guill\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     C:\Users\guill\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_eng is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\guill\AppData\Roaming\nltk_data...
[

True

In [49]:
!python -m spacy download en_core_web_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
      --------------------------------------- 0.3/12.8 MB ? eta -:--:--
      --------------------------------------- 0.3/12.8 MB ? eta -:--:--
     - ------------------------------------- 0.5/12.8 MB 578.7 kB/s eta 0:00:22
     - ------------------------------------- 0.5/12.8 MB 578.7 kB/s eta 0:00:22
     -- ------------------------------------ 0.8/12.8 MB 588.4 kB/s eta 0:00:21
     -- ------------------------------------ 0.8/12.8 MB 588.4 kB/s eta 0:00:21
     --- ----------------------------------- 1.0/12.8 MB 606.3 kB/s eta 0:00:20
     --- ----------------------------------- 1.0/12.8 MB 606.3 kB/s eta 0:00:20
     --- ----------------------------------- 1.3/12.8 MB 645.3 kB/s eta 0:00:18
     ---- ---------------------------------- 1.6/12.8 MB 655.4 kB/s eta 0:00:18


## 1. Introducción a NLP

### 1.1. Pregunta 1: ¿Qué es el Procesamiento de Lenguaje Natural y cuáles son sus principales aplicaciones?

### 1.1.1 Ejercicio: Investigar 3 aplicaciones actuales de NLP

### 1.2. Pregunta 2: Explica los conceptos de tokenización y segmentación de oraciones. ¿Por qué son importantes en NLP?

#### 1.2.1 Ejercicio: Usar la biblioteca nltk o spaCy en Python para tokenizar un texto corto.

##### 1.2.1.1 Ejercicio con Nltk

###### 1.2.1.1.1 Escribir un texto

In [50]:
nltk_text = "I am learning about tokenization!"

###### 1.2.1.1.2 Tokenizar por oraciones

In [51]:
sentences = sent_tokenize(nltk_text)
print(f"Sentance list: {sentences}.")

Sentance list: ['I am learning about tokenization!'].


###### 1.2.1.1.3 Tokenizar por palabras

In [52]:
words = word_tokenize(nltk_text)
print(f"Word list: {words}.")

Word list: ['I', 'am', 'learning', 'about', 'tokenization', '!'].


###### 1.2.1.1.4 Tokenizar por subpalabras

In [53]:
stemmer = PorterStemmer()
stemmer_subwords = [stemmer.stem(word) for word in words]
print(f"Subword list: {stemmer_subwords}.")

Subword list: ['i', 'am', 'learn', 'about', 'token', '!'].


In [54]:
lemmatizer = WordNetLemmatizer()
lemmatizer_subwords = [lemmatizer.lemmatize(word) for word in words]
print(f"Subword list: {lemmatizer_subwords}.")

Subword list: ['I', 'am', 'learning', 'about', 'tokenization', '!'].


###### 1.2.1.1.5 Tokenizar por caracteres

In [55]:
characters = list(nltk_text)
print(f"Character List: {characters}")

Character List: ['I', ' ', 'a', 'm', ' ', 'l', 'e', 'a', 'r', 'n', 'i', 'n', 'g', ' ', 'a', 'b', 'o', 'u', 't', ' ', 't', 'o', 'k', 'e', 'n', 'i', 'z', 'a', 't', 'i', 'o', 'n', '!']


#### 1.2.1.2 Ejercicio con spaCy

###### 1.2.1.2.1 Cargar el texto

In [57]:
nlp = spacy.load("en_core_web_sm")

In [56]:
spacy_text = "I am learning how to do tokenization with spacy. It is awesome!"

In [58]:
doc = nlp(spacy_text)

###### 1.2.1.2.2 Tokenizar por oraciones

In [59]:
sentences = []
for sentence in doc.sents:
    sentences.append(sentence)

sentences

[I am learning how to do tokenization with spacy., It is awesome!]

###### 1.2.1.2.2 Tokenizar por words

In [61]:
words = []
for token in doc:
    words.append(token.text)
words

['I',
 'am',
 'learning',
 'how',
 'to',
 'do',
 'tokenization',
 'with',
 'spacy',
 '.',
 'It',
 'is',
 'awesome',
 '!']

###### 1.2.1.2.3 Tokenizar por caracteres

In [60]:
characters = list(spacy_text)
characters

['I',
 ' ',
 'a',
 'm',
 ' ',
 'l',
 'e',
 'a',
 'r',
 'n',
 'i',
 'n',
 'g',
 ' ',
 'h',
 'o',
 'w',
 ' ',
 't',
 'o',
 ' ',
 'd',
 'o',
 ' ',
 't',
 'o',
 'k',
 'e',
 'n',
 'i',
 'z',
 'a',
 't',
 'i',
 'o',
 'n',
 ' ',
 'w',
 'i',
 't',
 'h',
 ' ',
 's',
 'p',
 'a',
 'c',
 'y',
 '.',
 ' ',
 'I',
 't',
 ' ',
 'i',
 's',
 ' ',
 'a',
 'w',
 'e',
 's',
 'o',
 'm',
 'e',
 '!']

## 2. Embeddings y Representaciones Vectoriales

### Ejercicio: Cargar un modelo de word embeddings de Hugging Face (por ejemplo, glove o word2vec) y visualizar los embeddings de palabras similares

### Ejercicio: Comparar GloVe (estático) con BERT (dinámico) mediante un ejemplo práctico en Python.

## 3. Modelos Basados en Redes Neuronales

### 3.1 Pregunta 5: ¿Qué son las redes neuronales recurrentes (RNN) y las LSTM, y cómo se usan en NLP?

#### 3.1.1 Ejercicio: Implementar una RNN simple para predicción de secuencias en Python.

### 3.2 Pregunta 6: ¿Qué son los transformers y por qué han reemplazado en gran medida a las RNN en NLP?

#### 3.2.1 Ejercicio: Utilizar el modelo de transformers BERT o GPT-2 de Hugging Face para completar una frase simple.

In [64]:
text = "Being a computer scientist is all about"

##### 3.2.1.1 Utilizar EleutherAI

In [65]:
eleutherAI_generator = pipeline("text-generation", model="EleutherAI/gpt-neo-125m")

In [66]:
set_seed(85)
eleutherAI_prediction = eleutherAI_generator(text)
eleutherAI_prediction

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
c:\Users\guill\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\generation\utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[{'generated_text': 'Being a computer scientist is all about the ability to analyze and understand the world. The world is a'}]

#### Utilizar GPT 2

In [67]:
set_seed(42)
gpt2_generator = pipeline("text-generation", model="gpt2") 

c:\Users\guill\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [68]:
gpt2_prediction = gpt2_generator(text)
gpt2_prediction

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "Being a computer scientist is all about getting what you want. But you're still a little frustrated when you can't make progress, and those things are what drive your problems: your frustration with your work, your frustration with technology.\n\nThere is"}]

## 4. Modelos Pre-entrenados y Fine-Tuning

### Pregunta 7: ¿Qué significa que un modelo esté preentrenado? ¿Cómo se puede ajustar para una tarea específica?

#### Ejercicio: Usar un modelo de Hugging Face (distilBERT) para clasificar un conjunto de datos de sentimientos (usar el dataset de Rotten tomatoes).

### Pregunta 8: ¿Qué es el enmascaramiento de palabras en el preentrenamiento de modelos como BERT?

#### Ejercicio: Usar BERT para predecir palabras enmascaradas en una oración

In [69]:
text = "Being a computer scientist is all about [MASK]."

In [70]:
bert_mask = pipeline("fill-mask", model="bert-base-uncased") 

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [71]:
bert_predition = bert_mask(text)
bert_predition

[{'score': 0.11361566185951233,
  'token': 7588,
  'token_str': 'computers',
  'sequence': 'being a computer scientist is all about computers.'},
 {'score': 0.06099285930395126,
  'token': 2671,
  'token_str': 'science',
  'sequence': 'being a computer scientist is all about science.'},
 {'score': 0.05559048056602478,
  'token': 2009,
  'token_str': 'it',
  'sequence': 'being a computer scientist is all about it.'},
 {'score': 0.048937588930130005,
  'token': 2470,
  'token_str': 'research',
  'sequence': 'being a computer scientist is all about research.'},
 {'score': 0.031744930893182755,
  'token': 8785,
  'token_str': 'math',
  'sequence': 'being a computer scientist is all about math.'}]

## 5. Tareas Clásicas de NLP

### 5.1 **Pregunta 9**: ¿Qué es la clasificación de texto y qué modelos se suelen utilizar para esta tarea?

#### Ejercicio: Implementar un modelo NER utilizando spaCy o Hugging Face y aplicar etiquetas a un conjunto de texto.

### 5.2 **Pregunta 10**: ¿Qué es el análisis de sentimientos y cómo se relaciona con la clasificación de texto?

#### Ejercicio: Utilizar Hugging Face para hacer análisis de sentimientos en un conjunto de datos de reseñas (usar el dataset de IMDB).

### 5.3 **Pregunta 11**: Explica el concepto de etiquetado de secuencias (named entity recognition, NER). ¿Para qué se utiliza?

#### 5.3.1 **Ejercicio**: Implementar un modelo NER utilizando spaCy o Hugging Face y aplicar etiquetas a un conjunto de texto.

In [14]:
text = """"
Trinamool Congress leader Mahua Moitra has moved the Supreme Court against her expulsion from the Lok Sabha over the cash-for-query allegations against her. Moitra was ousted from the Parliament last week after the Ethics Committee of the Lok Sabha found her guilty of jeopardising national security by sharing her parliamentary portal's login credentials with businessman Darshan Hiranandani."""

##### 5.3.1.1 Utilizar spaCy

In [16]:
import spacy

In [3]:
nlp = spacy.load("en_core_web_sm")

In [19]:
doc = nlp(text)

In [20]:
for word in doc.ents:
    print(f"This word {word.text} is this {word.label_} entity.")

This word Trinamool Congress is this ORG entity.
This word Mahua Moitra is this PERSON entity.
This word the Supreme Court is this ORG entity.
This word the Lok Sabha is this PERSON entity.
This word Moitra is this ORG entity.
This word Parliament is this ORG entity.
This word last week is this DATE entity.
This word the Ethics Committee is this ORG entity.
This word Darshan Hiranandani is this PERSON entity.


#### 3.1.1.2 Utilizar Hugging Face

In [11]:
from transformers import pipeline, set_seed

c:\Users\guill\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
bert_ner = pipeline("ner", model="dslim/bert-base-NER")

c:\Users\guill\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\guill\.cache\huggingface\hub\models--dslim--bert-base-NER. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializin

In [21]:
ner_results = bert_ner(text)
print(ner_results)

[{'entity': 'B-ORG', 'score': 0.9995658, 'index': 2, 'word': 'Tri', 'start': 2, 'end': 5}, {'entity': 'I-ORG', 'score': 0.99883586, 'index': 3, 'word': '##nam', 'start': 5, 'end': 8}, {'entity': 'I-ORG', 'score': 0.99924135, 'index': 4, 'word': '##ool', 'start': 8, 'end': 11}, {'entity': 'I-ORG', 'score': 0.99926585, 'index': 5, 'word': 'Congress', 'start': 12, 'end': 20}, {'entity': 'B-PER', 'score': 0.999546, 'index': 7, 'word': 'Ma', 'start': 28, 'end': 30}, {'entity': 'I-PER', 'score': 0.772231, 'index': 8, 'word': '##hua', 'start': 30, 'end': 33}, {'entity': 'I-PER', 'score': 0.9996074, 'index': 9, 'word': 'Mo', 'start': 34, 'end': 36}, {'entity': 'I-PER', 'score': 0.99346304, 'index': 10, 'word': '##it', 'start': 36, 'end': 38}, {'entity': 'I-PER', 'score': 0.97454643, 'index': 11, 'word': '##ra', 'start': 38, 'end': 40}, {'entity': 'B-ORG', 'score': 0.99936503, 'index': 15, 'word': 'Supreme', 'start': 55, 'end': 62}, {'entity': 'I-ORG', 'score': 0.99817765, 'index': 16, 'word': 

## 6. Generación de Texto

### 6.1 **Pregunta 12**: ¿Cómo funcionan los modelos de generación de texto como GPT-2 o GPT-3?

#### 6.1.1 **Ejercicio**: Usar Hugging Face para generar texto a partir de un prompt con GPT-2.

### 6.2 **Pregunta 13**: ¿Cuáles son los desafíos éticos asociados con los modelos de generación de texto?

#### 6.2.1 **Ejercicio**: Discutir sobre sesgos en los modelos de lenguaje generativo y su impacto en aplicaciones reales.

## 7. Evaluación de Modelos NLP

### 7.1 **Pregunta 14**: ¿Qué métricas se utilizan para evaluar la calidad de los modelos de NLP en tareas como clasificación o generación de texto?

#### 7.1.1 **Ejercicio**: Usar métricas como accuracy, precision, y recall para evaluar un modelo de clasificación de texto.

##### 7.1.1.1 Guardar las métricas de accuracy, precision y recall en una lista

In [98]:
metrics = ["accuracy", "recall", "precision", "f1"]

##### 7.1.1.2 Cargar los datos de imdb

In [99]:
data = load_dataset("imdb", split="test").shuffle(seed=42).select(range(1000))

##### 7.1.1.3 Cargar el evalvador para clasificación de textos.

In [100]:
task_evaluator = evaluator("text-classification")

##### 7.1.1.4 Cargar el modelo de clasificación de textos.

In [101]:
imdb_classification = pipeline("text-classification", model="lvwerra/distilbert-imdb")

c:\Users\guill\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


##### 7.1.1.5 Evaluar los resultados

In [106]:
eval_results = task_evaluator.compute(
    model_or_pipeline = imdb_classification,
    data = data,
    metric = evaluate.combine(metrics),
    label_mapping = {"NEGATIVE": 0, "POSITIVE": 1}
)
print(json.dumps(eval_results, indent=4))

{
    "accuracy": 0.918,
    "recall": 0.9180327868852459,
    "precision": 0.9142857142857143,
    "f1": 0.9161554192229039,
    "total_time_in_seconds": 119.51120690000243,
    "samples_per_second": 8.36741612723166,
    "latency_in_seconds": 0.11951120690000243
}


### 7.2 **Pregunta 15**: ¿Qué es la perplejidad y cómo se utiliza para evaluar modelos de generación de texto?

#### 7.2.1 **Ejercicio**: Calcular la perplejidad de un modelo de lenguaje en Hugging Face para una tarea de generación de texto.